In [1]:
from keras.applications import VGG16
from keras.models import load_model
from vis.utils import utils
from keras import activations

Using TensorFlow backend.


In [2]:
# Load our model
model = load_model('../datasets/cars_classification_best.hdf5')
#print(model.summary())

In [3]:
# Get the layer id we want to visualize from its name
layer_name = 'output'
layer_idx = utils.find_layer_idx(model, layer_name)
print(layer_idx)

22


In [4]:
import os
from keras.models import load_model

# Taken from vis package's util file and modified to work with Windows directory structure
def apply_modifications(model):
    """Applies modifications to the model layers to create a new Graph. For example, simply changing
    `model.layers[idx].activation = new activation` does not change the graph. The entire graph needs to be updated
    with modified inbound and outbound tensors because of change in layer building function.

    Args:
        model: The `keras.models.Model` instance.

    Returns:
        The modified model with changes applied. Does not mutate the original `model`.
    """
    # The strategy is to save the modified model and load it back. This is done because setting the activation
    # in a Keras layer doesnt actually change the graph. We have to iterate the entire graph and change the
    # layer inbound and outbound nodes with modified tensors. This is doubly complicated in Keras 2.x since
    # multiple inbound and outbound nodes are allowed with the Graph API.
    model_path = 'C:/Users/Richard/AppData/Local/Temp/temp.h5'
    try:
        model.save(model_path)
        return load_model(model_path)
    finally:
        os.remove(model_path)
        
# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = apply_modifications(model)

# Visualize saliency map and CAM

In [5]:
from matplotlib import pyplot as plt
%matplotlib inline
from vis.visualization import visualize_saliency, visualize_cam, overlay
from vis.utils import utils
from keras import activations
import numpy as np

plt.rcParams['figure.figsize'] = (18, 6)

# A demo image from each class
# Original images, IDs: 1426, 1362, 1392, 82, 962
#indices = ["1", "2", "5", "7", "8", "9", "12", "13", "15", "21", "24", "30", "31", "34", "44", "56", "73", "76", "78", "110", "111", "125", "128", "130", "135", "254", "277", "1214"]
#indices = np.load("cars_test_indices.npy")
indices = ["264"]
imgs = []
for c in indices:
    img = utils.load_img('../datasets/cars_im/'+str(c)+".jpg")
    imgs.append(img)
    # Reshape it to work with VGG16 
    #resized = np.copy(img).resize(3,224,224)
    #imgs_resized.append(resized)

In [6]:
for j in range(len(indices)):
    c = imgs[j]
    for i in range(5):
        plt.figure()
        grads = visualize_saliency(model, layer_idx, filter_indices=i, seed_input=c, backprop_modifier='guided')
        stitched = utils.stitch_images([c,grads], margin=0)
        plt.axis('off')
        plt.imshow(stitched)
        plt.savefig("../datasets/visualization/carsdemo/saliency/"+str(indices[j])+"_"+str(i)+".png")
        plt.close()

In [7]:
import numpy as np
import matplotlib.cm as cm

for j in range(len(indices)):
    c = imgs[j]
    for i in range(4):
        plt.figure()
        grads = visualize_cam(model, layer_idx, filter_indices=i, seed_input=c, backprop_modifier='guided')        
        # Create heatmap and overlay it on the original pic  
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
        plt.axis('off')
        plt.imshow(stitched)
        plt.savefig("../datasets/visualization/carsdemo/saliency/"+str(indices[j])+"_"+str(i)+".png")
        plt.close()